<a href="https://colab.research.google.com/github/Farhad-Davaripour/Navigating_GenAI/blob/main/prompt_engineering/prompt_engineering_demo.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Prompt Engineering

In [ ]:
%pip install openai scikit-learn
%pip install openai openai

In [1]:
import os
from openai import OpenAI 

OPENAI_API_KEY=""

# Set the environment variable
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI(
  api_key=OPENAI_API_KEY
)

In [9]:
def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Replace with ahe appropriate model name
        messages=prompt,
        temperature=0.8,
        max_tokens=500
    )
    return response

def print_response(prompt):
    response = generate_response(prompt)
    for key in response.choices[0]:
        print(key)
    print("\nResponse: ", response.choices[0].message.content)

## Example #1

In [13]:
# Generate response using the language model
def create_prompt_example1(prompt):
    system_prompt = """
        You are a bank customer service bot. 
        Your task is to assess customer intent and categorize customer 
        inquiries into one of the following predefined categories:
        
        - card arrival
        - change pin
        - exchange rate
        - country support 
        - cancel transfer
        - charge dispute
        
        If the text doesn't fit into any of the above categories, classify it as: customer service. 
        You will only respond with the predefined category. Do not provide explanations or notes.

        ###
        Here are some examples:
        
        Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
        Category: card arrival
        Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
        Category: exchange rate 
        Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
        Category: country support
        Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
        Category: customer service
        ###
    """
    
    user_prompt = f"""
        <<<
        Inquiry: {prompt}
        >>>
        Category:
    """
    
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_prompt
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt
                }
            ]
        },
    ]

In [15]:
prompt = create_prompt_example1("How do I know if I will get my card, or if it is lost?")
formatted_prompt = print_response(prompt)

('finish_reason', 'stop')
('index', 0)
('logprobs', None)
('message', ChatCompletionMessage(content='card arrival', role='assistant', function_call=None, tool_calls=None))

Response:  card arrival


## Example #2

### Zero Shot Learning

In [10]:
from IPython.display import Markdown, display

prompt = [{"role":"system","content":[{"type":"text","text":"You are an AI assistant that answers user's queries."}]},
          {"role":"user","content":[{"type":"text","text":"A car travels from point A to point B at an average speed of 60 km/h and returns from point B to point A at an average speed of 40 km/h. What is the average speed for the entire trip?"}]}]
formatted_prompt = print_response(prompt)
display(Markdown(formatted_prompt))

('finish_reason', 'stop')
('index', 0)
('logprobs', None)
('message', ChatCompletionMessage(content="To calculate the average speed for the entire trip, you can use the formula:\n\nAverage speed = total distance / total time\n\nLet's assume the distance between point A and point B is D km.\n\nFor the first leg of the trip from point A to point B:\nTime taken = D / 60\n\nFor the return trip from point B to point A:\nTime taken = D / 40\n\nTotal distance for the entire trip = 2D (since the car travels the same distance in both directions)\n\nTotal time for the entire trip = D / 60 + D / 40\n\nNow, substitute these values into the formula:\n\nAverage speed = 2D / (D / 60 + D / 40)\nAverage speed = 2D / (D * (1/60 + 1/40))\nAverage speed = 2 / (1/60 + 1/40)\nAverage speed = 2 / (1/24 + 1/24)\nAverage speed = 2 / (2/24)\nAverage speed = 2 / (1/12)\nAverage speed = 24 km/h\n\nTherefore, the average speed for the entire trip is 24 km/h.", role='assistant', function_call=None, tool_calls=None)

<IPython.core.display.Markdown object>

### COT and Few Shot Learning

In [11]:
def create_prompt_example2(prompt):
    system_prompt = """
        You are an AI assistant tasked with solving math problems step-by-step. 
        Your task is to provide a detailed explanation of each step in the solution process, ensuring that the user can follow along easily.

        ###
        Here are some examples:

        Problem: What is the sum of 8 and 12?
        Steps:
        1. Identify the numbers to be added: 8 and 12.
        2. Add the numbers: 8 + 12.
        3. Calculate the result: 20.
        Answer: 20

        Problem: How do you solve the equation 2x + 3 = 7?
        Steps:
        1. Identify the equation: 2x + 3 = 7.
        2. Subtract 3 from both sides: 2x + 3 - 3 = 7 - 3.
        3. Simplify: 2x = 4.
        4. Divide both sides by 2: 2x / 2 = 4 / 2.
        5. Simplify: x = 2.
        Answer: x = 2
        ###

    """
    
    user_prompt = f"""
        <<<
        Problem: {prompt}
        Steps:
    """
    
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_prompt
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt
                }
            ]
        },
    ]


In [12]:
from IPython.display import Markdown, display

prompt = create_prompt_example2("A car travels from point A to point B at an average speed of 60 km/h and returns from point B to point A at an average speed of 40 km/h. What is the average speed for the entire trip?")
formatted_prompt = print_response(prompt)
display(Markdown(formatted_prompt))

('finish_reason', 'stop')
('index', 0)
('logprobs', None)
('message', ChatCompletionMessage(content="Steps:\n1. Let's denote the distance between point A and point B as D (in kilometers).\n2. Calculate the time taken to travel from A to B:\n   - Time = Distance / Speed\n   - Time = D / 60  (since the speed is 60 km/h)\n   - Time = D / 60\n3. Calculate the time taken to travel from B to A:\n   - Time = Distance / Speed\n   - Time = D / 40  (since the speed is 40 km/h)\n   - Time = D / 40\n4. The total time for the entire trip (to and fro) is the sum of the time taken to travel from A to B and from B to A:\n   - Total Time = D / 60 + D / 40\n   - Total Time = (2D + 3D) / 120  (finding a common denominator)\n   - Total Time = 5D / 120\n5. Calculate the average speed for the entire trip:\n   - Average Speed = Total Distance / Total Time\n   - Average Speed = 2D / (5D / 120)  (as the total distance is 2D)\n   - Average Speed = 2D * 120 / 5D\n   - Average Speed = 240 / 5\n6. Simplify the ave

<IPython.core.display.Markdown object>